### Load tensorflow

In [0]:
import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(42)

### Read the data
<font size="2">Data for this exercise can be downloaded from http://www.manythings.org/anki/</font>

In [0]:
#You can use wget to download the file directly
!wget http://www.manythings.org/anki/hin-eng.zip --quiet

In [0]:
import zipfile
import io

#Read the zip file
zf = zipfile.ZipFile('hin-eng.zip', 'r')

#Extract data from zip file, here we can decide no of line to download as well. e.g. first 100 lines only
data = ''
with zf.open('hin.txt') as readfile:
  for line in io.TextIOWrapper(readfile, 'utf-8'):
    data += line

In [4]:
len(data)

181269

In [5]:
data[400:500]

"मज़े करो।\nI forgot.\tमैं भूल गया।\nI forgot.\tमैं भूल गई।\nI'll pay.\tमैं पैसे दूंगा।\nI'm fine.\tमैं ठीक ह"


### Extract Source and Target Language pairs

In [6]:
#Split by newline character
data =  data.split('\n')

#Show some Data
data[100:105]

["I don't know.\tमुझे नहीं पता।",
 "I don't know.\tमुझे नहीं मालूम।",
 'I have a car.\tमेरे पास एक गाड़ी है।',
 'I have a dog.\tमेरे पास एक कुत्ता है।',
 'I understand.\tमैं समझता हूँ।']

In [7]:
# No of sentences
len(data)

2809

### Separate Source and Target pairs

In [0]:
encoder_text = [] #Initialize Source language list
decoder_text = [] #Initialize Target language list

#Iterate over data
for line in data:
    try:
        in_txt, out_txt = line.split('\t')
        encoder_text.append(in_txt)
        
        # Add tab '<start>' as 'start sequence in target
        # And '<end>' as End
        decoder_text.append('<start> ' + out_txt + ' <end>')
    except:
        pass #ignore data which goes into error        

### Separate Source and Target pairs..

In [9]:
encoder_text[100:105]

["I don't know.",
 "I don't know.",
 'I have a car.',
 'I have a dog.',
 'I understand.']

In [10]:
decoder_text[100:105]

['<start> मुझे नहीं पता। <end>',
 '<start> मुझे नहीं मालूम। <end>',
 '<start> मेरे पास एक गाड़ी है। <end>',
 '<start> मेरे पास एक कुत्ता है। <end>',
 '<start> मैं समझता हूँ। <end>']

### Tokenize Source language sentences

In [11]:
#Tokenizer for source language
encoder_t = tf.keras.preprocessing.text.Tokenizer()
encoder_t.fit_on_texts(encoder_text) #Fit it on Source sentences
encoder_seq = encoder_t.texts_to_sequences(encoder_text) #Convert sentences to numbers 
encoder_seq[100:105] #Display some converted sentences

[[2, 27, 43], [2, 27, 43], [2, 14, 6, 99], [2, 14, 6, 125], [2, 210]]

In [14]:
# Encoding dictionary. Word to index
encoder_t.word_index

{'the': 1,
 'i': 2,
 'to': 3,
 'you': 4,
 'is': 5,
 'a': 6,
 'he': 7,
 'of': 8,
 'in': 9,
 'my': 10,
 'it': 11,
 'me': 12,
 'this': 13,
 'have': 14,
 'she': 15,
 'for': 16,
 'was': 17,
 'are': 18,
 'do': 19,
 'that': 20,
 'his': 21,
 'your': 22,
 'we': 23,
 'will': 24,
 'on': 25,
 'what': 26,
 "don't": 27,
 'at': 28,
 'him': 29,
 'her': 30,
 'not': 31,
 'like': 32,
 'go': 33,
 'with': 34,
 'be': 35,
 'how': 36,
 'and': 37,
 "i'm": 38,
 'can': 39,
 'time': 40,
 'has': 41,
 'there': 42,
 'know': 43,
 'all': 44,
 'up': 45,
 'they': 46,
 'come': 47,
 'very': 48,
 'as': 49,
 'had': 50,
 'please': 51,
 'from': 52,
 'want': 53,
 "it's": 54,
 'here': 55,
 'by': 56,
 'out': 57,
 'am': 58,
 'when': 59,
 'did': 60,
 'no': 61,
 'been': 62,
 'get': 63,
 "can't": 64,
 'take': 65,
 'going': 66,
 'an': 67,
 'father': 68,
 'book': 69,
 'if': 70,
 'about': 71,
 'one': 72,
 'india': 73,
 'were': 74,
 'money': 75,
 'would': 76,
 'tom': 77,
 'day': 78,
 'long': 79,
 'two': 80,
 'tomorrow': 81,
 'today': 82

In [15]:
#Maximum length of sentence
max_encoder_seq_length = max([len(txt) for txt in encoder_seq])
print('Maximum sentence length for Source language: ', max_encoder_seq_length)

#Source language Vocablury
encoder_vocab_size = len(encoder_t.word_index)
print('Source language vocablury size: ', encoder_vocab_size)

Maximum sentence length for Source language:  22
Source language vocablury size:  2388


### Tokenize Target language sentences

In [0]:
#Tokenizer for target language, filters should not <start> and <end>
#remove < and > used in Target language sequences
decoder_t = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
decoder_t.fit_on_texts(decoder_text) #Fit it on target sentences
decoder_seq = decoder_t.texts_to_sequences(decoder_text) #Convert sentences to numbers 

In [19]:
decoder_text[100:105]

['<start> मुझे नहीं पता। <end>',
 '<start> मुझे नहीं मालूम। <end>',
 '<start> मेरे पास एक गाड़ी है। <end>',
 '<start> मेरे पास एक कुत्ता है। <end>',
 '<start> मैं समझता हूँ। <end>']

In [20]:
decoder_seq[100:105] #Display some converted sentences

[[1, 11, 5, 622, 2],
 [1, 11, 5, 1488, 2],
 [1, 28, 40, 19, 103, 3, 2],
 [1, 28, 40, 19, 202, 3, 2],
 [1, 6, 769, 22, 2]]

In [32]:
decoder_t.word_index

{'<start>': 1,
 '<end>': 2,
 'है।': 3,
 'में': 4,
 'नहीं': 5,
 'मैं': 6,
 'वह': 7,
 'से': 8,
 'के': 9,
 'क्या': 10,
 'मुझे': 11,
 'है': 12,
 'को': 13,
 'हैं।': 14,
 'की': 15,
 'हो': 16,
 'बहुत': 17,
 'का': 18,
 'एक': 19,
 'पर': 20,
 'उसने': 21,
 'हूँ।': 22,
 'तुम': 23,
 'था।': 24,
 'यह': 25,
 'कर': 26,
 'लिए': 27,
 'मेरे': 28,
 'कि': 29,
 'और': 30,
 'हैं': 31,
 'इस': 32,
 'उसे': 33,
 'भी': 34,
 'अपने': 35,
 'मेरी': 36,
 'मैंने': 37,
 'रहा': 38,
 'ने': 39,
 'पास': 40,
 'तो': 41,
 'करने': 42,
 'अपनी': 43,
 'कल': 44,
 'हम': 45,
 'गया।': 46,
 'ही': 47,
 'काम': 48,
 'कोई': 49,
 'उसके': 50,
 'तुम्हारे': 51,
 'उसकी': 52,
 'उस': 53,
 'तुम्हें': 54,
 'था': 55,
 'साथ': 56,
 'घर': 57,
 'करना': 58,
 'थी।': 59,
 'समय': 60,
 'मेरा': 61,
 'गया': 62,
 'सकते': 63,
 'आप': 64,
 'बात': 65,
 'रही': 66,
 'पता': 67,
 'कुछ': 68,
 'यहाँ': 69,
 'आज': 70,
 'अभी': 71,
 'बजे': 72,
 'तक': 73,
 'तुम्हे': 74,
 'उसको': 75,
 'चाहिए।': 76,
 'लगता': 77,
 'किताब': 78,
 'दिया।': 79,
 'मुझसे': 80,
 'पसंद': 81,
 'अच्छा': 82,

In [21]:
#Maximum length of sentence
max_decoder_seq_length = max([len(txt) for txt in decoder_seq])
print('Maximum sentence length for Target language: ', max_decoder_seq_length)

#Target language Vocablury
decoder_vocab_size = len(decoder_t.word_index)
print('Target language vocablury size: ', decoder_vocab_size)

Maximum sentence length for Target language:  27
Target language vocablury size:  2992


### Compare different sentences length

In [22]:
#Source Language sentences
print('Length for sentence number 100: ', len(encoder_seq[100]))
print('Length for sentence number 2000: ', len(encoder_seq[2000]))

Length for sentence number 100:  3
Length for sentence number 2000:  7


In [0]:
# word at index no 100 has lenghth 3 and word at index no 2000 has lenghth 7

In [23]:
#Target Language sentences
print('Length for sentence number 100: ', len(decoder_seq[100]))
print('Length for sentence number 2000: ', len(decoder_seq[2000]))

Length for sentence number 100:  5
Length for sentence number 2000:  13


In [0]:
# word at index no 100 has lenghth 5 and word at index no 2000 has lenghth 13

### How do we make it same?

### Padding the sentences

In [0]:
#Source sentences
encoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(encoder_seq, 
                                                                   maxlen=max_encoder_seq_length, #22
                                                                   padding='pre')

#Target Sentences
decoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(decoder_seq, 
                                                                   maxlen=max_decoder_seq_length, #27
                                                                   padding='post')

In [25]:
print('Source data shape: ', encoder_input_data.shape)
print('Target data shape: ', decoder_input_data.shape)

Source data shape:  (2808, 22)
Target data shape:  (2808, 27)


In [33]:
encoder_text[20]

'Have fun.'

In [34]:
encoder_input_data[20]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  14, 277], dtype=int32)

In [35]:
decoder_text[20]

'<start> मज़े करना। <end>'

In [36]:
decoder_input_data[20]

array([  1, 615, 978,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0], dtype=int32)

#### Integer to Word converter for Decoder data

In [0]:
int_to_word_decoder = dict((i,c) for c, i in decoder_t.word_index.items())

In [38]:
int_to_word_decoder[15]

'की'

### Building Decoder Output

In [39]:
decoder_input_data.shape

(2808, 27)

In [0]:
import numpy as np

#Initialize array
decoder_target_data = np.zeros((decoder_input_data.shape[0], decoder_input_data.shape[1]))

#Shift Target output by one word
for i in range(decoder_input_data.shape[0]):
    for j in range(1,decoder_input_data.shape[1]):
        decoder_target_data[i][j-1] = decoder_input_data[i][j]

In [0]:
#decoder_t.word_index

In [0]:
#<start> yeh kitab hai <end>
#Yeh kitab hai <end> 0

In [43]:
decoder_input_data[0]

array([  1, 757,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0], dtype=int32)

In [44]:
decoder_target_data[0]

array([757.,   2.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.])

#### Convert target data in one hot vector

In [0]:
#Initialize one hot encoding array
decoder_target_one_hot = np.zeros((decoder_input_data.shape[0], #number of sentences
                                   decoder_input_data.shape[1], #Number of words in each sentence
                                   len(decoder_t.word_index)+1)) #Vocab size + 1

In [46]:
decoder_target_one_hot.shape

(2808, 27, 2993)

In [0]:
#Build one hot encoded array
for i in range(decoder_target_data.shape[0]):
    for j in range(decoder_target_data.shape[1]):
        decoder_target_one_hot[i][j] = tf.keras.utils.to_categorical(decoder_target_data[i][j],
                                                                     num_classes=len(decoder_t.word_index)+1)    

In [48]:
decoder_target_one_hot.shape

(2808, 27, 2993)

### Building the Training Model

In [0]:
#Define config parameters
encoder_embedding_size = 50
decoder_embedding_size = 50
rnn_units = 256

#### Build Encoder

In [50]:
#Input Layer
encoder_inputs = tf.keras.layers.Input(shape=(None,))

#Embedding layer
encoder_embedding = tf.keras.layers.Embedding(encoder_vocab_size+1, encoder_embedding_size)

#Get embedding layer output by feeding inputs
encoder_embedding_output = encoder_embedding(encoder_inputs)

#LSTM Layer and its output
x, state_h, state_c = tf.keras.layers.LSTM(rnn_units,return_state=True)(encoder_embedding_output)

#Build a list to feed Decoder - Sentence Embedding
encoder_states = [state_h, state_c]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [51]:
encoder_embedding_output

<tf.Tensor 'embedding/embedding_lookup/Identity_1:0' shape=(?, ?, 50) dtype=float32>

In [0]:
#  shape=(?, ?, 50)  it is an tensor object

#### Build Decoder

In [0]:
#Decode input - padded Target sentences
decoder_inputs = tf.keras.layers.Input(shape=(None,))

#Decoder Embedding layer
decoder_embedding = tf.keras.layers.Embedding(decoder_vocab_size + 1, decoder_embedding_size)

#Embedding layer output
decoder_embedding_output = decoder_embedding(decoder_inputs)

#Decoder RNN
decoder_rnn = tf.keras.layers.LSTM(rnn_units, return_sequences=True, return_state=True)

#Decoder RNN Output, State initialization from Encoder states
#Output will be all hidden sequences, last 'h' state and last 'c' state
x,_,_ = decoder_rnn(decoder_embedding_output, initial_state=encoder_states)

#Output Layer
decoder_dense = tf.keras.layers.Dense(decoder_vocab_size + 1, activation='softmax')

#Output of Dense layer
decoder_outputs = decoder_dense(x)

In [54]:
decoder_outputs

<tf.Tensor 'dense/truediv:0' shape=(?, ?, 2993) dtype=float32>

### Build Model using both Encoder and Decoder

In [0]:
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], #2 Inputs to the model
                              decoder_outputs) #Output of the model

In [56]:
model.output

<tf.Tensor 'dense/truediv:0' shape=(?, ?, 2993) dtype=float32>

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

### Train the model

In [58]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_one_hot,
          batch_size=64,
          epochs=2,
          validation_split=0.2)

Train on 2246 samples, validate on 562 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/2
2246/2246 [==============================] - 12s 5ms/sample - loss: 3.7466 - val_loss: 2.9813
Epoch 2/2
2246/2246 [==============================] - 6s 2ms/sample - loss: 1.6439 - val_loss: 2.9954


### Save the model for later reuse

In [0]:
model.save('seq2seq_training_translation.hd5')

# Building Model for Prediction

### Build the Encoder Model to predict Encoder States

In [0]:
encoder_model = tf.keras.models.Model(encoder_inputs, #Padded input sequences
                                      encoder_states) #Hidden state and Cell state at last time step

### Build the Decoder Model 
<p/>

<ol><li>Define Input for both 'h' state and 'c' state initialization </li>
<li>Get Decoder RNN outputs along with h and c state</li>
<li>Get Decoder Dense layer output</li>
        <li>Build Model</li></ol>

##### Step 1 - Define Input for both 'h' state and 'c' state initialization

In [0]:
#Hidden state input
decoder_state_input_h = tf.keras.layers.Input(shape=(rnn_units,))

#Cell state input
decoder_state_input_c = tf.keras.layers.Input(shape=(rnn_units,))

#Putting it together
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

##### Step 2 - Get Decoder RNN outputs along with h and c state

In [0]:
#Get Embedding layer output
x = decoder_embedding(decoder_inputs)

#We will use the layer which we trained earlier
rnn_outputs, state_h, state_c = decoder_rnn(x, initial_state=decoder_states_inputs)

#Why do we need this?
decoder_states = [state_h, state_c]

##### Step 3 - Get Decoder Dense layer output

In [0]:
decoder_outputs = decoder_dense(rnn_outputs)

##### Step 4 - Build Decoder Model

In [0]:
decoder_model = tf.keras.models.Model([decoder_inputs] + decoder_states_inputs,  #Model inputs
                                      [decoder_outputs] + decoder_states)

# Predicting output from Seq2Seq model

##### Build a prediction function

In [65]:
decoder_t.word_index['<start>']

1

In [66]:
int_to_word_decoder[1]

'<start>'

In [0]:
def decode_sentence(input_sequence):
    
    #Get the encoder state values - Sentence embedding
    decoder_initial_states_value = encoder_model.predict(input_seq)
    
    #Build a sequence with '<start>' - starting sequence for Decoder
    target_seq = np.zeros((1,1))    
    target_seq[0][0] = decoder_t.word_index['<start>']
    
    #flag to check if prediction should be stopped
    stop_loop = False
    
    #Initialize predicted sentence
    predicted_sentence = ''
    
    num_of_predictions = 0
    
    #start the loop
    while not stop_loop:
        
        predicted_outputs, h, c = decoder_model.predict([target_seq] + 
                                                        decoder_initial_states_value)
        
        #Get the predicted word index with highest probability
        predicted_output = np.argmax(predicted_outputs[0,-1,:])
        
        #Get the predicted word from predicter index
        predicted_word = int_to_word_decoder[predicted_output]
        
        #Check if prediction should stop
        if(predicted_word == '<end>' or num_of_predictions > max_decoder_seq_length):
            
            stop_loop = True
            continue
        
        num_of_predictions += 1
        
        #Updated predicted sentence
        if (len(predicted_sentence) == 0):
            predicted_sentence = predicted_word
        else:
            predicted_sentence = predicted_sentence + ' ' + predicted_word
            
        #Update target_seq to be the predicted word index
        target_seq[0][0] = predicted_output
        
        #Update initial states value for decoder
        decoder_initial_states_value = [h,c]
        
    
    return predicted_sentence

##### Call Prediction function on a random sentence

In [68]:
#Generate a random number
start_num = np.random.randint(0, high=len(encoder_text) - 10)

#Predict model output for 5 sentences
for i in range(start_num, start_num + 5):
    input_seq = encoder_input_data[i : i+1]
    predicted_sentence = decode_sentence(input_seq)
    print('--------')
    print ('Input sentence: ', encoder_text[i])
    print ('Predicted sentence: ', predicted_sentence )

--------
Input sentence:  I met him at the barber's.
Predicted sentence:  वह वह
--------
Input sentence:  I sold it for ten dollars.
Predicted sentence:  वह वह
--------
Input sentence:  I usually get up at eight.
Predicted sentence:  वह वह
--------
Input sentence:  I want to see the manager.
Predicted sentence:  वह वह
--------
Input sentence:  I want to see the manager.
Predicted sentence:  वह वह


In [0]:
# For every input it is predicting 'वह वह' which is incorrect. 
# We have to train data with more epochs and with more train data to get more accuracy.

##### Save encoder and decoder model

In [0]:
#Compile models to avoid error
encoder_model.compile(optimizer='adam',loss='categorical_crossentropy')
decoder_model.compile(optimizer='adam',loss='categorical_crossentropy')

#Save the models
encoder_model.save('seq2seq_encoder_eng_hin.hd5')  #Encoder model
decoder_model.save('seq2seq_decoder_eng_hin.hd5')  #Decoder model

##### Save encoder and decoder tokenizers

In [0]:
import pickle

pickle.dump(encoder_t,open('encoder_tokenizer_eng','wb'))
pickle.dump(decoder_t,open('decoder_tokenizer_hin','wb'))